In [1]:
import polars as pl

### Read the data

In [2]:
calendar = (
    pl.read_csv(
        "../data/calendar.csv",
        dtypes={
            "date": pl.Date(),
            "weekday": pl.Categorical(),
            "month": pl.Categorical(),
            "event_name_1": pl.Categorical(),
            "event_type_1": pl.Categorical(),
            "event_name_2": pl.Categorical(),
            "event_type_2": pl.Categorical(),
            "snap_CA": pl.Categorical(),
            "snap_TX": pl.Categorical(),
            "snap_WI": pl.Categorical(),
        }
    )
    .rename({"d": "date_index"})
)

sales = pl.read_csv(
    "../data/sales_train_validation.csv",
    dtypes={
        "id": pl.Categorical(),
        "item_id": pl.Categorical(),
        "store_id": pl.Categorical(),
        "dept_id": pl.Categorical(),
        "cat_id": pl.Categorical(),
        "state_id": pl.Categorical(),
    }
)

### Preprocess the data

In [3]:
df = (
    sales
    .melt(
        id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
        value_name="sales"
    )
    .rename({"variable": "date_index"})
    .join(calendar, on="date_index", how="left", validate="m:1")
    .filter(pl.col("date").dt.year().is_in((2012, 2013)))
    .with_columns(
        (pl.col("date").dt.year() + pl.col("date").dt.ordinal_day() / 365).alias("time_linear")
    )
)

assert df.group_by("item_id", "store_id", "date_index").count().filter(pl.col("count") > 1).is_empty()

target = "sales"
numerical_features = ["time_linear"]
categorical_features = [
    "item_id",
    "store_id",
    "dept_id",
    "cat_id",
    "state_id",
    "weekday",
    "month",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "snap_CA",
    "snap_TX",
    "snap_WI"
]

df = (
    df.select(target, "date", *numerical_features, *categorical_features)
    .sort("date", "store_id", "item_id")
)

In [4]:
assert df.group_by("store_id", "item_id", "date").count().filter(pl.col("count") > 1).is_empty()

### Add lags 

The names of the features indicate for which model (i.e., for which _horizon_) the features are available.
In doing so, we assume that if you forecast on November 8 for November 9 (i.e., horizon=1), we have observed the sales on November 8.

For example, `lag_1___max_horizon_1` simply indicates that this feature is only available for models for a horizon of at most 1 (e.g. if you make a forecast on Monday for Wednesday, you cannot use sales of Tuesday because it did not yet realize).  

In [5]:
df = (
    df.with_columns(
        
        # plain vanilla lags ...
        *(pl.col("sales").shift(i).over(["store_id", "item_id"]).alias(f"lag_{i}___max_horizon_{i}") for i in range(1, 8)),
        
        # and lags based on the day of the week
        *(pl.col("sales").shift(i).over(["store_id", "item_id", "weekday"]).alias(f"lag_dow_{i}___max_horizon_{i * 7 - 1}") for i in [1, 2, 3])
        
    )
)

### Add moving averages

For example, the feature "ma_7d___max_horizon_2" is a moving average of seven days that is only available to models with a horizon of at most 2 (so, if the date is 2012-01-09, then the corresponding feature is the average of the sales from 2012-01-01 until 2012-01-07, which is available on 2012-01-07 --- given that we assumed that we forecast at the end of the day).


In [6]:
from datetime import timedelta

for period in (7, 14):
    for lag in (1, 7, 14):
        
        moving_averages = (
            df.rolling("date", period=f"{period}d", offset=f"-{period+lag}d", by=["item_id", "store_id"])
            .agg(pl.col("sales").mean().alias(f"ma_{period}d___max_horizon_{lag}"))
        )
        
        df = df.join(moving_averages, how="left", on=["store_id", "item_id", "date"]).sort("date")

### Add day-of-week based moving averages

E.g. "ma_dow_14d___max_horizon_7" means a moving average of the last two corresponding day of weeks shifted by a week.
So on Thursday 2013-07-25, it's the average of Thursday 2013-07-18 and 2013-07-11.


In [7]:
from datetime import timedelta

for period in (7, 14, 28):
    for lag in (7, 14):
        
        moving_averages = (
            df.rolling("date", period=f"{period}d", offset=f"-{period+lag}d", by=["item_id", "store_id", "weekday"])
            .agg(pl.col("sales").mean().alias(f"ma_dow_{period}d___max_horizon_{lag}"))
        )
        
        df = df.join(moving_averages, how="left", on=["store_id", "item_id", "weekday", "date"]).sort("date")

In [8]:
df.write_parquet("../data/features.parquet")